In [1]:
import os
import string
import tempfile
import tensorflow as tf
import numpy as np
import pandas as pd
from tensorflow import keras
from tensorflow.python.keras.preprocessing import sequence
from tensorboard import summary as summary_lib
from pymystem3 import Mystem
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

from sklearn.utils import resample
from sklearn.model_selection import train_test_split

import string
import razdel
from sklearn.feature_extraction.text import CountVectorizer
from tqdm import tqdm_notebook as tqdm

tf.logging.set_verbosity(tf.logging.INFO)
print(tf.__version__)

Using TensorFlow backend.


1.12.0


### Работа с данными (1 балл)

Загрузите датасет, с которым вы работали во время соревнования на kaggle. Преобразуйте его в формат, удобный для обучения модели. В качетсве фичей используйте эмбединги слов.

In [2]:
tr = str.maketrans(string.punctuation, ' '*len(string.punctuation))

def tokkenize_razdel(text):
    return [token.text for token in razdel.tokenize(text)]
 
def token(text):
    result = []
    for doc in text:    
        doc = tokkenize_razdel(doc)
        result.append(doc)        
    return result

In [3]:
tokkenize_razdel('SSTARTT')

['SSTARTT']

In [4]:
# tf.reset_default_graph()

data = pd.read_csv('train.csv', index_col= False)

Y = np.array(data.label)
X = np.array(data.text)

x_train_variable = " SSTARTT " + X
y_train = Y

In [5]:
pad_id = 0
oov_id = 'UNK'
index_offset = 2
sentence_size = 30
vocab_size = 5000
embedding_size = 20
model_dir = tempfile.mkdtemp()

def preprocess_vectorize(data):
    data = token(data)
    
    for element in data:
        element = ' '.join(element)

    tokenizer = Tokenizer(num_words = vocab_size, 
                          filters ='!"$%&()*+,-./:;=?@[\\]^_`{|}~\t\n', 
                          lower=True, 
                          oov_token= oov_id)
    tokenizer.fit_on_texts(data)
    data = tokenizer.texts_to_sequences(data)
    
    return data

In [6]:
%%time
x_train_variable = preprocess_vectorize(x_train_variable)

CPU times: user 16.1 s, sys: 163 ms, total: 16.2 s
Wall time: 16.3 s


In [7]:
X_train_variable, X_val_variable, Y_train, Y_val = train_test_split(x_train_variable, y_train, test_size=0.2, random_state=42)
print("X, Y for train:", len(X_train_variable), ",", Y_train.size)
print("X, Y for validation:", len(X_val_variable),",", Y_val.size)

X, Y for train: 89973 , 89973
X, Y for validation: 22494 , 22494


In [8]:
X_train = pad_sequences(X_train_variable, maxlen= sentence_size, padding = 'post',truncating = 'post')
X_val = pad_sequences(X_val_variable, maxlen= sentence_size, padding = 'post',truncating = 'post')

In [9]:
x_len_train = np.array([min(len(x), sentence_size) for x in X_train_variable])
x_len_test = np.array([min(len(x), sentence_size) for x in X_val_variable])

def parser(x, length, y):
    features = {"x": x, "len": length}
    return features, y

def train_input_fn():
    dataset = tf.data.Dataset.from_tensor_slices((X_train, x_len_train, Y_train))
    dataset = dataset.shuffle(buffer_size=len(X_train_variable))
    dataset = dataset.batch(50)
    dataset = dataset.map(parser)
    dataset = dataset.repeat()
    iterator = dataset.make_one_shot_iterator()
    return iterator.get_next()

def eval_input_fn():
    dataset = tf.data.Dataset.from_tensor_slices((X_val, x_len_test, Y_val))
    dataset = dataset.batch(50)
    dataset = dataset.map(parser)
    iterator = dataset.make_one_shot_iterator()
    return iterator.get_next()

In [10]:
column = tf.feature_column.categorical_column_with_identity('x', vocab_size)
classifier = tf.estimator.LinearClassifier(feature_columns=[column], model_dir=os.path.join(model_dir, 'bow_sparse'),n_classes=3)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/var/folders/zw/2w6s_cjx4h9dj9fjz5t8tr800000gn/T/tmpb37d5_l5/bow_sparse', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x13c42ca20>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [11]:
all_classifiers = {}
def train_and_evaluate(classifier):
    # Save a reference to the classifier to run predictions later
    all_classifiers[classifier.model_dir] = classifier
    classifier.train(input_fn=train_input_fn, steps=1500)
    eval_results = classifier.evaluate(input_fn=eval_input_fn)
    predictions = np.array([p['class_ids'][0] for p in classifier.predict(input_fn=eval_input_fn)])
        
    # Reset the graph to be able to reuse name scopes
    tf.reset_default_graph() 
    # Add a PR summary in addition to the summaries that the classifier writes
#     pr = summary_lib.pr_curve('precision_recall', predictions=predictions, labels=y_test.astype(bool), num_thresholds=21)
    with tf.Session() as sess:
        writer = tf.summary.FileWriter(os.path.join(classifier.model_dir, 'eval'), sess.graph)
#         writer.add_summary(sess.run(pr), global_step=0)
        writer.close()

In [12]:
train_and_evaluate(classifier)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into /var/folders/zw/2w6s_cjx4h9dj9fjz5t8tr800000gn/T/tmpb37d5_l5/bow_sparse/model.ckpt.
INFO:tensorflow:loss = 54.930614, step = 1
INFO:tensorflow:global_step/sec: 172.535
INFO:tensorflow:loss = 32.252804, step = 101 (0.580 sec)
INFO:tensorflow:global_step/sec: 1074.5
INFO:tensorflow:loss = 21.3676, step = 201 (0.093 sec)
INFO:tensorflow:global_step/sec: 1062.61
INFO:tensorflow:loss = 24.701527, step = 301 (0.094 sec)
INFO:tensorflow:global_step/sec: 1063.23
INFO:tensorflow:loss = 18.72807, step = 401 (0.094 sec)
INFO:tensorflow:global_step/sec: 1003.7
INFO:tensorflow:loss = 15.614145, step = 501 (0.100 sec)
INFO:tensorflow:global_step/sec: 1015.38
INFO:tensorflow:loss = 10.980184, step = 601 (0.099 sec)
INFO:te

### CNN model (1 балл)

Реализуйте модель со следующими слоями, идущими в указанном порядке:
- Conv1D, activation='relu'
- MaxPooling1D
- Dense

Параметры модели подберите сами.

In [13]:
column = tf.feature_column.categorical_column_with_identity('x', vocab_size)
word_embedding_column = tf.feature_column.embedding_column(column, dimension=embedding_size)

In [18]:
head = tf.contrib.estimator.multi_label_head(n_classes=3)

def cnn_model_fn(features, labels, mode, params):    
    input_layer = tf.contrib.layers.embed_sequence(
        features['x'], vocab_size, embedding_size,
        initializer=params['embedding_initializer'])
    
    training = mode == tf.estimator.ModeKeys.TRAIN


    conv = tf.layers.conv1d(
        inputs=input_layer,
        filters=32,
        kernel_size=3,
        padding="same",
        activation=tf.nn.relu)
    
    # Global Max Pooling
    pool = tf.reduce_max(input_tensor=conv, axis=1)
    
    hidden = tf.layers.dense(inputs=pool, units=250, activation=tf.nn.relu)
    

    
    logits = tf.layers.dense(inputs=hidden, units=3)
    
    optimizer = tf.train.AdamOptimizer()
    
    def _train_op_fn(loss):
        return optimizer.minimize(
            loss=loss,
            global_step=tf.train.get_global_step())
    print("EXEXEXEXE")

    labels = tf.one_hot(labels, 3)
  
    return head.create_estimator_spec(
        features=features,
        labels=labels,
        mode=mode,
        logits=logits, 
        train_op_fn=_train_op_fn)
  
params = {'embedding_initializer': tf.random_uniform_initializer(-1.0, 1.0)}
cnn_classifier = tf.estimator.Estimator(model_fn=cnn_model_fn,
                                        model_dir=os.path.join(model_dir, 'cnn'),
                                        params=params)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/var/folders/zw/2w6s_cjx4h9dj9fjz5t8tr800000gn/T/tmpb37d5_l5/cnn', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x1435d2ba8>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [19]:
cnn_classifier.train(input_fn=train_input_fn, steps=10000 )
eval_results = cnn_classifier.evaluate(input_fn=eval_input_fn)
prediction = cnn_classifier.predict(input_fn=eval_input_fn)

INFO:tensorflow:Calling model_fn.
EXEXEXEXE
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /var/folders/zw/2w6s_cjx4h9dj9fjz5t8tr800000gn/T/tmpb37d5_l5/cnn/model.ckpt-20000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 20000 into /var/folders/zw/2w6s_cjx4h9dj9fjz5t8tr800000gn/T/tmpb37d5_l5/cnn/model.ckpt.
INFO:tensorflow:loss = 0.12920953, step = 20001
INFO:tensorflow:global_step/sec: 110.123
INFO:tensorflow:loss = 0.14024536, step = 20101 (0.909 sec)
INFO:tensorflow:global_step/sec: 290.67
INFO:tensorflow:loss = 0.08002748, step = 20201 (0.344 sec)
INFO:tensorflow:global_step/sec: 298.525
INFO:tensorflow:loss = 0.13519835, step = 20301 (0.335 sec)
INFO:tensorflow:global_step/sec: 271.921
INFO:tensorflow:loss = 0.052825946, step = 20401 (0.368 sec)
INFO:tensorflow:global_step/sec: 221.657
INFO:tens

INFO:tensorflow:global_step/sec: 292.046
INFO:tensorflow:loss = 0.06679263, step = 27701 (0.342 sec)
INFO:tensorflow:global_step/sec: 284.938
INFO:tensorflow:loss = 0.14652485, step = 27801 (0.351 sec)
INFO:tensorflow:global_step/sec: 300.572
INFO:tensorflow:loss = 0.06638595, step = 27901 (0.333 sec)
INFO:tensorflow:global_step/sec: 290.705
INFO:tensorflow:loss = 0.07143498, step = 28001 (0.344 sec)
INFO:tensorflow:global_step/sec: 290.308
INFO:tensorflow:loss = 0.024092149, step = 28101 (0.344 sec)
INFO:tensorflow:global_step/sec: 299.761
INFO:tensorflow:loss = 0.044760257, step = 28201 (0.334 sec)
INFO:tensorflow:global_step/sec: 300.141
INFO:tensorflow:loss = 0.06301949, step = 28301 (0.333 sec)
INFO:tensorflow:global_step/sec: 292.979
INFO:tensorflow:loss = 0.10463678, step = 28401 (0.341 sec)
INFO:tensorflow:global_step/sec: 302.178
INFO:tensorflow:loss = 0.051904365, step = 28501 (0.331 sec)
INFO:tensorflow:global_step/sec: 303.544
INFO:tensorflow:loss = 0.03164601, step = 28601

### RNN model (1 балл)

Реализуйте модель со следующими слоями, идущими в указанном порядке:
- RNN
- Dense

Параметры модели подберите сами.

### LSTM model (1 балл)

Реализуйте модель со следующими слоями, идущими в указанном порядке:
- LSTM
- Dense

Параметры модели подберите сами.

In [ ]:
# your code here

Сравните все три реализованные модели по времени и по качеству классификации. Какая лучше? Как думаете, почему?

<b>Вывод:</b>

### Baseline model (2 балла)

Реализуйте модель со следующими слоями, идущими в указанном порядке:
- Conv1D, activation='relu'
- MaxPooling1D
- RNN
- Dense

Параметры для Conv1D, MaxPooling1D и RNN подберите сами.

In [ ]:
# your code here

### Dropout (1 балл)

Добавьте dropout к baseline моделе. Пример модефицированной модели:
- Dropout
- Conv1D, activation='relu'
- MaxPooling1D
- RNN
- Dense

Подберите параметры.

In [ ]:
# your code here

Улучшило ли результат классификации использование Dropout? Как думаете, почему?

<b>Вывод:</b>

### Bidirectional (1 балл)

Вместо RNN в первой модели используйте biRNN. Пример модефецированной модели:
- Conv1D, activation='relu'
- MaxPooling1D
- biRNN
- Dense

Подберите параметры.

In [ ]:
# your code here

Дало ли буст использовании biRNN вместо RNN? Сильный? Как думаете, почему?

<b>Вывод:</b>

### Custom model (2 балла)

Подберите архитектуру сети, используя сверточные слои и слои макспулинга и превзойдите качество полносвязной сети.

Подберите архитектуру сети, используя следующие типы слоев:
- Conv1D
- MaxPooling1D
- RNN
- LSTM
- GRU
- Dropout
- Dense

Можно использовать любые из описанных слоев в любом количестве и в любом порядке. Не обязательно использовать все описанные слои. 

Настройте параметры. Превзойдите лучшее качество, полученное раннее. 

In [ ]:
# your code here